## Python 中的装饰器

装饰器本质上是一个Python函数，**它可以让其他函数在不需要做任何代码变动的前提下增加额外功能**，装饰器的返回值是一个函数对象。装饰器可以减少重复代码，抽离出与函数功能无关的代码并继续重用，减少重复工作。

### 简单装饰器

In [1]:
import time


def call_time(func):
    def _wrapper(arg):
        start = time.time()
        result = func(arg)
        print('used time: {}'.format(time.time() - start))
        return result

    return _wrapper

In [2]:
def func_1(n):
    print("func_1 function is running...")
    for i in range(n):
        time.sleep(0.01)

In [3]:
func_1 = call_time(func_1)
func_1(5)

func_1 function is running...
used time: 0.05299854278564453


@ 符号的使用

In [4]:
@call_time
def func_2(n):
    print("func_2 function is running...")
    for i in range(n):
        time.sleep(0.01)

In [5]:
func_2(10)

func_2 function is running...
used time: 0.10700011253356934


修饰器提高了程序的可重复利用性和可读性

### 带参数的修饰器

装饰器的语法允许我们调用的时候提供其他参数

In [6]:
import logging


def use_logging(level):
    def decorator(func):
        def _wrapper(arg):
            if level == 'warn':
                logging.warn("warn logging")
            return func(arg)

        return _wrapper

    return decorator

In [7]:
@use_logging(level='warn')
def func_3(n):
    print(n)
    print("func_3 is running")

In [8]:
func_3(3)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  


3
func_3 is running


### functools.wraps

使用装饰器极大地复用了代码，但是存在一个缺点：原函数的元信息不见了，比如函数的docstring, 参数列表等

In [9]:
@call_time
def func_4(n):
    """
    @param n: is the number of customers
    @return int: the customers value point
    """
    for i in range(n):
        print(n)
        time.sleep(0.1)
    return 0

In [10]:
func_4(3)

3
3
3
used time: 0.30301403999328613


0

In [11]:
help(func_4)

Help on function _wrapper in module __main__:

_wrapper(arg)



函数被\_wrapper取代了，它的元信息就变成了_wrapper函数的信息了

In [12]:
print(func_4.__name__)

_wrapper


functools.wraps本身也是一个装饰器，它能把原函数的元信息拷贝到装饰器函数中，使得装饰器函数和原函数具有一样的信息

In [13]:
from functools import wraps
from collections import defaultdict

function_called_time = defaultdict(int)


def call_time_2(func):
    @wraps(func)
    def _wrapper(arg):
        """It's inner function"""
        global function_called_time
        function_called_time[func.__name__] += 1
        result = func(arg)
        print('function called time: {}'.format(
            function_called_time[func.__name__]))
        return result

    return _wrapper

In [14]:
@call_time_2
def func_5(n):
    """
    @param n: is the number of customers
    @return int: the customers value point
    """
    for i in range(n):
        print(n)
        time.sleep(0.1)
    return 0

In [15]:
func_5(3)

3
3
3
function called time: 1


0

In [16]:
help(func_5)

Help on function func_5 in module __main__:

func_5(n)
    @param n: is the number of customers
    @return int: the customers value point



### \*args 和\*\*kwargs 两个魔法变量

可以将不定数量的参数传递给一个函数。

- \*args = list of arguments - as positional arguments
- \*\*kwargs = dictionary - whose keys become separate keyword arguments and the values become values of these arguments.

In [17]:
def test_var_args(f_arg, *argv):
    print("first normal arg:", f_arg)
    for arg in argv:
        print("another arg through *argv:", arg)

test_var_args('yasoob', 'python', 'eggs', 'test')

first normal arg: yasoob
another arg through *argv: python
another arg through *argv: eggs
another arg through *argv: test


In [18]:
test_var_args('yasoob', 'eggs', 'test')

first normal arg: yasoob
another arg through *argv: eggs
another arg through *argv: test


In [19]:
def greet_me(**kwargs):
    for key, value in kwargs.items():
        print("{0} == {1}".format(key, value))

In [20]:
greet_me(name="python")

name == python


In [21]:
greet_me(name="python", age=18)

name == python
age == 18


In [22]:
kwargs = {"arg3": 3, "arg2": "two", "arg1": 5}
greet_me(**kwargs)

arg3 == 3
arg2 == two
arg1 == 5


## Dynamic Programming

In [23]:
from collections import defaultdict

In [24]:
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30]
price = defaultdict(int)
for i, p in enumerate(original_price):
    price[i + 1] = p
assert price[1] == 1

In [25]:
def memo(func):
    cache = {}

    @wraps(func)
    def _wrap(n):
        if n in cache:
            result = cache[n]
        else:
            result = func(n)
            cache[n] = result
        return result

    return _wrap

In [26]:
solution = {}

In [27]:
@memo
def r(n):
    max_price, split_point = max(
        [(price[n], 0)] + [(r(i) + r(n - i), i) for i in range(1, n)],
        key=lambda x: x[0])
    solution[n] = (split_point, n - split_point)
    return max_price

In [28]:
r(200)

600

In [29]:
solution[20]

(10, 10)

### 对Solution进行解析

In [30]:
def not_cut(split):
    return split == 0


def parse_solution(target_length, revenue_solution):
    left, right = revenue_solution[target_length]
    if not_cut(left):
        return [right]
    return parse_solution(left, revenue_solution) + parse_solution(
        right, revenue_solution)

In [31]:
parse_solution(25, solution)

[2, 3, 10, 10]

## Edit Distance

是指两个字串之间，由一个转成另一个所需的最少编辑操作次数，如果它们的距离越大，说明它们越是不同。编辑操作只有三种**插入，删除，替换**

In [32]:
solution = {}

In [33]:
from functools import lru_cache


@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):

    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)

    tail_s1 = string1[-1]
    tail_s2 = string2[-1]

    condidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),
    ]

    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1,
                        'SUB {} => {}'.format(tail_s1, tail_s2))
    condidates.append(both_forward)

    mindistance, operation = min(condidates, key=lambda x: x[0])
    solution[(string1, string2)] = operation
    return mindistance

In [34]:
edit_distance('ABCDE', 'ABCCEF')

2

In [35]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F'}

### Parse Solution

In [36]:
def edit(string, opreation):
    if opreation == '':
        return string[:-1]
    elif opreation[:3] == 'ADD':
        return string + opreation[-1]
    elif opreation[:3] == 'DEL':
        return string[-1]
    elif opreation[:3] == 'SUB':
        return string[-1] + opreation[-1]


def not_edit(string1, string2):
    return string1 == string2


def parse_solution(string1, string2, revenue_solution):

    if not_edit(string1, string2):
        return []

    opreation = revenue_solution.get((string1, string2))
    new_string1 = edit(string1, opreation)

    if new_string1 == string2:
        return [opreation]

    if opreation == '':
        return parse_solution(string1[:-1], string2[:-1], revenue_solution)
    elif opreation[:3] == 'ADD':
        
        return parse_solution(string1, string2[:-1], revenue_solution) + [opreation]
    elif opreation[:3] == 'DEL':
        return parse_solution(string1[:-1], string2, revenue_solution) + [opreation]
    elif opreation[:3] == 'SUB':
        return parse_solution(string1[:-1], string2[:-1], revenue_solution) + [opreation]

In [37]:
parse_solution('ABCDE', 'ABCCEF', solution)

['SUB D => C', 'ADD F']

In [38]:
edit_distance('怎么今天又熬夜了', '我真的不想熬夜啊啊啊啊')

9

In [39]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F',
 ('怎', '我'): 'SUB 怎 => 我',
 ('怎', '我真'): 'ADD 真',
 ('怎', '我真的'): 'ADD 的',
 ('怎', '我真的不'): 'ADD 不',
 ('怎', '我真的不想'): 'ADD 想',
 ('怎', '我真的不想熬'): 'ADD 熬',
 ('怎', '我真的不想熬夜'): 'ADD 夜',
 ('怎', '我真的不想熬夜啊'): 'ADD 啊',
 ('怎', '我真的

In [40]:
parse_solution('怎么今天又熬夜了', '我真的不想熬夜啊啊啊啊', solution)

['SUB 怎 => 我',
 'SUB 么 => 真',
 'SUB 今 => 的',
 'SUB 天 => 不',
 'SUB 又 => 想',
 'SUB 了 => 啊',
 'ADD 啊',
 'ADD 啊',
 'ADD 啊']